In [1]:
import torch
import torch.nn as nn
from pysmt.shortcuts import Symbol, And, Equals, Real, GT, Max, Plus, get_model
from pysmt.typing import REAL

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()             
        self.linear0 = nn.Linear(3, 4)
        self.linear1 = nn.Linear(4, 1)
        self.linear_layers = [self.linear0, self.linear1]
        self.relu = nn.ReLU()
        
    def forward(self, x):   
        x = self.linear0(x)
        x = self.relu(x)  
        
        x = self.linear1(x)
        x = self.relu(x) 
        
        return x[0]

In [3]:
def train_backpropagation(model, all_data, labels, criterion, optimizer, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_correct = 0
    
        for (data, label) in zip(all_data, labels): 
            optimizer.zero_grad()
        
            output = model(data)
            prediction = 1 if output > 0 else 0

            loss = criterion(output, label)
            num_correct += (label == prediction)
        
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

        loss = running_loss/len(labels)
        accuracy = num_correct/len(labels)
        print("epoch: " + str(epoch) + ", accuracy: " + str(accuracy.item()) + ", loss: " + str(loss))
        
        if (accuracy == 1.0):
            return

In [4]:
def train_smt(model, all_data, labels):
    
    layers_weights = []
    num_layers = len(model.linear_layers)
    for i in range(num_layers):
        num_rows = len(model.linear_layers[i].weight.data)
        num_cols = len(model.linear_layers[i].weight.data[0])
        layer_weights = []
        for j in range(num_rows):
            weights = []
            for k in range(num_cols):
                weights.append(Symbol("weight_" + str(i) + "_" + str(j) + "_" + str(k), REAL))
            weights.append(Symbol(str("bias_" + str(i) + "_" + str(j)), REAL))
            layer_weights.append(weights)
        layers_weights.append(layer_weights)

    equations = []
    for (data, label) in zip(all_data, labels):
        x = []
        for value in data:
            x.append(Real(value.item()))
            
        for i in range(num_layers):
            num_rows = len(model.linear_layers[i].weight.data)
            num_cols = len(model.linear_layers[i].weight.data[0])
            new_x = []
            for j in range(num_rows):
                calculations = []
                for k in range(num_cols):
                    calculation = layers_weights[i][j][k] * x[k]
                    calculations.append(calculation)
                calculations.append(layers_weights[i][j][num_cols])
                calculation = Plus(calculations)
                relu = Max(calculation, Real(0.0))
                new_x.append(relu)
            x = new_x
    
        if label.item() == 0.0:
            equation = Equals(x[0], Real(0.0))
        else:
            equation = GT(x[0], Real(0.0))
            
        equations.append(equation)

    formula = And(equations)
    solution = get_model(formula)
    
    if solution:
        print(solution)
        
        for i in range(num_layers):
            num_rows = len(model.linear_layers[i].weight.data)
            num_cols = len(model.linear_layers[i].weight.data[0])
            for j in range(num_rows):
                for k in range(num_cols):
                    weight = layers_weights[i][j][k]
                    weight_solution = solution[weight].constant_value()
                    model.linear_layers[i].weight.data[j][k] = float(weight_solution)
    
                
                bias = layers_weights[i][j][num_cols]
                bias_solution = solution[bias].constant_value()
                model.linear_layers[i].bias.data[j] = float(bias_solution)

    else:
        print("No solution found")

In [5]:
def test(model, all_data, labels, criterion): 
    running_loss = 0.0
    num_correct = 0
    
    for (data, label) in zip(all_data, labels): 
        output = model(data)
        prediction = 1 if output > 0 else 0

        loss = criterion(output, label)
        num_correct += (label == prediction)
        
        running_loss += loss.item()

    loss = running_loss/len(labels)
    accuracy = num_correct/len(labels)
    
    print("testing accuracy: " + str(accuracy.item()) + " and loss: " + str(loss))

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model()
model.to(device)

points = torch.tensor([[-1.0, -1.0, -1.0], [1.0, -1.0, -1.0], [-1.0, 1.0, -1.0], [-1.0, -1.0, 1.0],
                       [1.0, 1.0, 1.0], [-1.0, 1.0, 1.0], [1.0, -1.0, 1.0], [1.0, 1.0, -1.0],])

labels = torch.tensor([0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0], requires_grad=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
num_epochs = 200

In [7]:
train_backpropagation(model, points, labels, criterion, optimizer, num_epochs)
test(model, points, labels, criterion)

epoch: 0, accuracy: 0.625, loss: 0.7192254513502121
epoch: 1, accuracy: 0.625, loss: 0.697275921702385
epoch: 2, accuracy: 0.75, loss: 0.6762716174125671
epoch: 3, accuracy: 0.625, loss: 0.6802159771323204
epoch: 4, accuracy: 0.625, loss: 0.675903208553791
epoch: 5, accuracy: 0.75, loss: 0.6621194630861282
epoch: 6, accuracy: 0.75, loss: 0.6437512822449207
epoch: 7, accuracy: 0.75, loss: 0.6235737688839436
epoch: 8, accuracy: 0.875, loss: 0.5905947163701057
epoch: 9, accuracy: 0.75, loss: 0.5789399519562721
epoch: 10, accuracy: 0.875, loss: 0.5459853000938892
epoch: 11, accuracy: 0.875, loss: 0.5266023278236389
epoch: 12, accuracy: 1.0, loss: 0.50243472866714
testing accuracy: 0.75 and loss: 0.5022241063416004


In [8]:
train_smt(model, points, labels)
test(model, points, labels, criterion)

weight_0_1_0 := 1.0
weight_0_0_0 := 1/8
weight_0_2_2 := 1/8
weight_1_0_2 := -5.0
bias_0_0 := -1/2
weight_0_0_1 := 1/2
weight_0_2_0 := -1/2
weight_0_2_1 := -1.0
weight_1_0_0 := 1.0
weight_0_3_0 := -1/8
weight_0_0_2 := -1/2
bias_1_0 := 61/64
weight_0_1_1 := -1.0
weight_1_0_3 := -9/16
weight_0_3_1 := -1.0
bias_0_1 := -1/2
bias_0_2 := -1/4
bias_0_3 := 3.0
weight_0_1_2 := 1/2
weight_1_0_1 := 2.0
weight_0_3_2 := -1/4
testing accuracy: 1.0 and loss: 0.6198121905326843
